In [2]:
import polars as pl
import lightgbm as lgb
from typing import Tuple, List, Dict
import os
from sklearn.metrics import mean_absolute_error
import numpy as np
from numpy.typing import NDArray
from tqdm import tqdm
from polars.dataframe.group_by import GroupBy

import tensorflow as tf
from tensorflow.keras.layers import (
    Input,
    Dense,
    Embedding,
    Flatten,
    Concatenate,
    GaussianNoise,
)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.experimental import CosineDecay
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.layers import concatenate, Dropout
import pickle
from tensorflow.keras.models import load_model
import os
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.callbacks import Callback
import random
from tensorflow.keras.layers import (
    Input,
    Embedding,
    Lambda,
    Reshape,
    LSTM,
    Dense,
    BatchNormalization,
    Dropout,
    concatenate,
)
from tensorflow.keras import backend as K
from tensorflow.keras.layers import ZeroPadding1D
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import RepeatVector

from tensorflow.keras.layers import (
    Input,
    Embedding,
    Lambda,
    Reshape,
    LSTM,
    Dense,
    BatchNormalization,
    Dropout,
    concatenate,
)
from tensorflow.keras import backend as K
from tensorflow.keras.layers import ZeroPadding1D, Activation
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import MaxPooling1D, AveragePooling1D
from tensorflow.keras.layers import Add

import datetime
import pandas as pd
import json
import os

In [3]:
from enum import Enum
class Runtime(Enum):
    LOCAL = 0
    COLAB = 1
    KAGGLE = 2

In [4]:
runtime = Runtime.COLAB

In [5]:
if runtime == Runtime.COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
dates_train = (0, 400)
dates_test = (401, 480)
num_models = {"rnn": 1}
models_path = "gru_models"

In [7]:
rnn_ep = 100
rnn_lr = 0.001
rnn_bs = 2**12
window_size = 3

In [8]:
def split_by_date(df: pl.DataFrame, dates: Tuple[int, int]) -> pl.DataFrame:
    return df.filter(
        pl.col("date_id").ge(dates[0]).and_(pl.col("date_id").le(dates[1]))
    )


def make_predictions(models, X_test, model="nn"):
    if model == "nn":
        all_predictions = [model.predict(X_test, batch_size=16384) for model in models]
    if model == "lgb" or model == "xgb" or model == "cat":
        all_predictions = [model.predict(X_test) for model in models]
    prediction = np.mean(all_predictions, axis=0)
    return prediction

In [9]:
# @title TPU
try:
    # Create a TPUClusterResolver
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    # Connect to the TPU cluster
    tf.config.experimental_connect_to_cluster(tpu)
    # Initialize the TPU system
    tf.tpu.experimental.initialize_tpu_system(tpu)
    # Create a TPUStrategy for distributed training
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    tpu_strategy = None  # No TPU found

In [10]:

from typing import List
from itertools import combinations
from typing import Tuple

# @title Feature Engineering Functions


def lag_function(
    df: pl.DataFrame, columns_to_lag: List[str], num_days_to_lag: List[int]
) -> pl.DataFrame:
    cols = [
        pl.col(columns_to_lag)
        .shift(i)
        .over(["stock_id", "seconds_in_bucket"])
        .name.prefix(f"lag{i}_")
        for i in num_days_to_lag
    ]
    return df.with_columns(*cols)


def create_diff_lagged_features_within_date(
    df: pl.DataFrame, columns_to_lag: List[str], lags: List[int]
) -> pl.DataFrame:
    cols = [
        pl.col(columns_to_lag)
        .sub(pl.col(columns_to_lag).shift(lag).over(["stock_id", "date_id"]))
        .name.suffix(f"_lag_{lag}")
        for lag in lags
    ]
    return df.with_columns(*cols)


def create_features_from_start(df: pl.DataFrame, columns: List[str]) -> pl.DataFrame:
    return df.with_columns(
        pl.col(columns)
        .sub(pl.col(columns).first().over(["stock_id", "date_id"]))
        .name.suffix("_from_start")
    )


def compute_imbalances(df: pl.DataFrame, columns: List[str], prefix="") -> pl.DataFrame:
    cols = []
    for col1, col2 in combinations(columns, 2):
        col1, col2 = sorted([col1, col2])
        total = pl.col(col1).add(pl.col(col2))
        col_name = f"{col1}_{col2}_imbalance_{prefix}"
        cols.append(pl.col(col1).sub(pl.col(col2)).truediv(total).alias(col_name))
    return df.with_columns(*cols)





In [11]:
#@title pipeline

raw_cols          = ['imbalance_size','matched_size','bid_size','ask_size','reference_price','far_price','near_price','bid_price','ask_price','wap','imbalance_buy_sell_flag']

columns_prices    = ['reference_price','far_price','near_price','bid_price','ask_price','wap']
columns_4prices   = ['reference_price','bid_price','ask_price','wap']

columns_sizes     = ['imbalance_size','matched_size','bid_size','ask_size']
columns_flag      = ['imbalance_buy_sell_flag']


num_of_target_lags  = 3
target_lags         = list(range(1,num_of_target_lags+1))

def feature_pipeline(df: pl.DataFrame):

    df = compute_imbalances(df, columns_sizes, prefix='_sz_')
    df = compute_imbalances(df, columns_prices, prefix = '_pr_')


    print(f"lagging target column for {len(target_lags)} lags.")
    df = lag_function(df, ['target'], target_lags)

    print("Done...")

    return df

In [12]:
from typing import Any


from numpy import dtype, ndarray


def set_all_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

class BestScoresCallback(Callback):
    def __init__(self):
        super().__init__()
        self.best_train_loss = float('inf')
        self.best_val_loss = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            train_loss = logs.get('loss', float('inf'))
            val_loss = logs.get('val_loss', float('inf'))

            if train_loss < self.best_train_loss:
                self.best_train_loss = train_loss
            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss

    def on_train_end(self, logs=None):
        print(f"Best training loss: {self.best_train_loss}, Best validation loss: {self.best_val_loss}")

# @title RNN second pass


def precompute_sequences(
    stock_data: pl.DataFrame,
    window_size: int,
    rnn_numerical_features: List[str],
    rnn_categorical_features: List[str],
) -> Tuple[ndarray[Any, dtype[Any]], ndarray[Any, Any]]:
    # Convert DataFrame columns to NumPy arrays
    stock_data_num = stock_data.select(rnn_numerical_features).to_numpy()
    stock_data_cat = stock_data.select(rnn_categorical_features).to_numpy()

    # Pre-compute all sequences
    all_sequences_num = [
        stock_data_num[max(0, i - window_size + 1) : i + 1]
        for i in range(len(stock_data))
    ]
    all_sequences_cat = [
        stock_data_cat[max(0, i - window_size + 1) : i + 1]
        for i in range(len(stock_data))
    ]

    # Add padding if necessary
    padded_sequences_num = [
        np.pad(seq, ((window_size - len(seq), 0), (0, 0)), "constant")
        for seq in all_sequences_num
    ]
    padded_sequences_cat = [
        np.pad(seq, ((window_size - len(seq), 0), (0, 0)), "constant")
        for seq in all_sequences_cat
    ]

    # Combine numerical and categorical features
    combined_sequences = np.array(
        [
            np.concatenate([num, cat], axis=-1)
            for num, cat in zip(padded_sequences_num, padded_sequences_cat)
        ]
    )

    # Extract targets
    targets = stock_data.select("target").to_numpy()

    return combined_sequences, targets


def get_sequence(precomputed_data: Tuple[NDArray, NDArray], time_step):
    combined_sequences, targets = precomputed_data
    return combined_sequences[time_step], targets[time_step]


def create_batches(
    data: pl.DataFrame,
    window_size: int,
    rnn_numerical_features: List[str],
    rnn_categorical_features: List[str],
    max_time_steps: int = 55,
):

    grouped = data.group_by(["stock_id", "date_id"], maintain_order=True)
    all_batches = []
    all_targets = []

    for _, group in tqdm(grouped, desc="Processing groups"):
        # Precompute sequences for the current group
        precomputed_data = precompute_sequences(
            group, window_size, rnn_numerical_features, rnn_categorical_features
        )

        # Initialize containers for group sequences and targets
        group_sequences = []
        group_targets = []

        # Iterate over the time steps and retrieve precomputed sequences
        for time_step in range(max_time_steps):
            sequence, target = get_sequence(precomputed_data, time_step)
            if sequence.size > 0:
                group_sequences.append(sequence)
                group_targets.append(target)

        # Extend the main batches with the group's sequences and targets
        all_batches.extend(group_sequences)
        all_targets.extend(group_targets)

    return all_batches, all_targets


def compute_last_sequence(
    group: pl.DataFrame, window_size: int, rnn_numerical_features: List[str], rnn_categorical_features: List[str]
):
    # Convert DataFrame columns to NumPy arrays
    # group_all = group.all()
    stock_data_num = group.select(rnn_numerical_features).to_numpy()
    stock_data_cat = group.select(rnn_categorical_features).to_numpy()
    stock_data_target = group.select("target").to_numpy()

    # Find the index of the target second
    target_index = len(group) - 1

    # Extract the sequence for the target index
    sequence_num = stock_data_num[
        max(0, target_index - window_size + 1) : target_index + 1
    ]
    sequence_cat = stock_data_cat[
        max(0, target_index - window_size + 1) : target_index + 1
    ]

    # Add padding if necessary
    padded_sequence_num = np.pad(
        sequence_num, ((window_size - len(sequence_num), 0), (0, 0)), "constant"
    )
    padded_sequence_cat = np.pad(
        sequence_cat, ((window_size - len(sequence_cat), 0), (0, 0)), "constant"
    )

    # Combine numerical and categorical features
    combined_sequence = np.concatenate(
        [padded_sequence_num, padded_sequence_cat], axis=-1
    )

    # Extract target
    target = stock_data_target[-1]

    return combined_sequence, target


def create_last_batches(
    data: pl.DataFrame, window_size: int, rnn_numerical_features: List[str], rnn_categorical_features: List[str]
):

    grouped = data.group_by(["stock_id"], maintain_order=True)
    all_batches = []
    all_targets = []

    for _, group in grouped:
        # Compute the sequence for the last data point in the current group
        last_sequence, last_target = compute_last_sequence(
            group, window_size, rnn_numerical_features, rnn_categorical_features
        )

        # Check if the sequence is valid (i.e., not empty)
        if last_sequence.size > 0:
            all_batches.append(last_sequence)
            all_targets.append(last_target)

    return all_batches, all_targets


def second_pass_for_rnn(
    df: pl.DataFrame,
    rnn_numerical_features: List[str],
    rnn_categorical_features: List[str],
    window_size: int,
    rnn_scaler: StandardScaler,
    rnn_medians: Dict[str, float],
    is_inference=False,
):

    # Standard scaling for numerical features
    if is_inference:
        df = df.with_columns(
            [pl.col(col).fill_null(rnn_medians[col]) for col in rnn_numerical_features]
        )
        df = df.with_columns(
            [
                pl.Series(
                    rnn_scaler.fit_transform(df.select(pl.col(col)).to_numpy()).flatten()
                ).alias(col)
                for col in rnn_numerical_features
            ]
        )

    # Preprocess Data
    df = df.with_columns(
        [
            pl.col("seconds_in_bucket").truediv(10).alias("seconds_in_bucket"),
            pl.col("imbalance_buy_sell_flag").add(1).alias("imbalance_buy_sell_flag"),
        ]
    )

    if is_inference:
        df_copy_batches, df_copy_targets = create_last_batches(
            df, window_size, rnn_numerical_features, rnn_categorical_features
        )
    else:
        df_copy_batches, df_copy_targets = create_batches(
            df, window_size, rnn_numerical_features, rnn_categorical_features
        )

    del df
    import gc
    gc.collect()

    df_copy_batches = np.array(df_copy_batches)
    df_copy_targets = np.array(df_copy_targets)

    return df_copy_batches, df_copy_targets


# ------------------------------------------------- NN Second Pass Functions -----------------------------------------------

In [13]:
#@title RNN model
from tensorflow.keras.layers import Input, Embedding, Lambda, Reshape, LSTM, GRU, Dense, BatchNormalization, Dropout, concatenate
from tensorflow.keras import backend as K
from tensorflow.keras.layers import ZeroPadding1D


def create_rnn_model_with_residual(window_size, numerical_features, initial_learning_rate=0.001):

    categorical_features = 'seconds_in_bucket'
    categorical_uniques  = { 'seconds_in_bucket' : 55}
    embedding_dim        = {'seconds_in_bucket' : 10}

    input_layer = Input(shape=(window_size, len(numerical_features) + 1), name="combined_input")

    # Split the input into numerical and categorical parts
    numerical_input = Lambda(lambda x: x[:, :, :-1], name="numerical_part")(input_layer)
    categorical_input = Lambda(lambda x: x[:, :, -1:], name="categorical_part")(input_layer)

    # Function to create a difference layer for a given lag
    def create_difference_layer(lag):
        return Lambda(lambda x: x[:, lag:, :] - x[:, :-lag, :], name=f"difference_layer_lag{lag}")

    # List to store all difference layers
    difference_layers = []

    # Create difference layers for each lag
    for lag in range(1, window_size):
        diff_layer = create_difference_layer(lag)(numerical_input)
        padding = ZeroPadding1D(padding=(lag, 0))(diff_layer)  # Add padding to the beginning of the sequence
        difference_layers.append(padding)



    combined_diff_layer = Concatenate(name="combined_difference_layer")(difference_layers)

    enhanced_numerical_input = Concatenate(name="enhanced_numerical_input")([numerical_input, combined_diff_layer])

#     concat_input = Concatenate(name="concatenated_input")([enhanced_numerical_input, categorical_input])

    # Embedding for categorical part
    vocab_size, embedding_dim = categorical_uniques[categorical_features], embedding_dim[categorical_features]
    embedding = Embedding(vocab_size, embedding_dim, input_length=window_size)(categorical_input)
    embedding = Reshape((window_size, -1))(embedding)



    # Concatenate numerical input and embedding
    gru_input = concatenate([enhanced_numerical_input, embedding], axis=-1)

    # Initialize a list to hold the outputs of each LSTM layer
#     lstm_outputs = []

    # First LSTM layer
    grul = GRU(64, return_sequences=False)(gru_input)
    grul = BatchNormalization()(grul)
    grul = Dropout(0.3)(grul)

    dense_output = grul
    dense_sizes = [512, 256, 128, 64, 32]
    do_ratio = 0.3
    for size in dense_sizes:
        dense_output = Dense(size, activation='swish')(dense_output)
        dense_output = BatchNormalization()(dense_output)
        dense_output = Dropout(do_ratio)(dense_output)

    # Output layer
    output = Dense(1, name='output_layer')(dense_output)

    # Learning rate schedule
    lr_schedule = ExponentialDecay(
        initial_learning_rate=initial_learning_rate,
        decay_steps=10000,
        decay_rate=0.7,
        staircase=True)

    # Create and compile the model
    model = Model(inputs=input_layer, outputs=output)
    optimizer = Adam(learning_rate=lr_schedule)

    model.compile(optimizer=optimizer, loss="mean_absolute_error")

    return model


In [14]:
excluded_columns = [
    "row_id",
    "date_id",
    "time_id",
    "target",
    "stock_return",
    "stock_id",
]

match runtime:
    case Runtime.LOCAL:
        root_dir = "."
    case Runtime.COLAB:
        root_dir = "/content/drive/MyDrive/optiver"
    case Runtime.KAGGLE:
        root_dir = "/kaggle/input/optiver-train"

train_eng = pl.read_parquet(f"{root_dir}/data/train_eng_rnn.parquet")
features = [col for col in train_eng.schema.keys() if col not in excluded_columns]
categorical_features = ["seconds_in_bucket"]
numerical_features = [
    col for col in features if col not in categorical_features
]
print("we have {} features".format(len(features)))


scaler = StandardScaler()
medians = train_eng.select([pl.col(f).median() for f in numerical_features]).to_dict(
    as_series=False
)
medians = {k: v[0] for k, v in medians.items()}


train_eng = train_eng.with_columns(
    [pl.col(f).fill_null(medians[f]) for f in numerical_features]
)

train_eng = train_eng.with_columns(
    [pl.when(pl.col(f).is_nan()).then(medians[f]).otherwise(pl.col(f)).alias(f) for f in numerical_features]
)

train_eng = train_eng.with_columns(
    [
        pl.Series(
            scaler.fit_transform(train_eng.select(pl.col(f)).to_numpy()).flatten()
        ).alias(f)
        for f in numerical_features
    ]
)


train_data = split_by_date(train_eng, dates_train)
test_data = split_by_date(train_eng, dates_test)
stock_ids = test_data.select("stock_id").to_numpy().flatten()


del train_eng
import gc

gc.collect()

train_batches, train_targets = second_pass_for_rnn(
    train_data,
    numerical_features,
    categorical_features,
    window_size,
    rnn_scaler=scaler,
    rnn_medians=medians,
)
test_batches, test_targets = second_pass_for_rnn(
    test_data,
    numerical_features,
    categorical_features,
    window_size,
    rnn_scaler=scaler,
    rnn_medians=medians,
)
print(f"train batches shape:{train_batches.shape}")

we have 87 features


Processing groups: 79236it [07:25, 177.98it/s]
Processing groups: 16000it [01:28, 180.29it/s]


train batches shape:(4357980, 3, 87)


In [15]:
log_dir = f"{root_dir}/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
# callbacks = [BestScoresCallback()]  # Always include BestScoresCallback
callbacks = []
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
callbacks.append(early_stopping)

if dates_train[1] != 480:
    early_stopping = EarlyStopping(
        monitor="val_loss", patience=20, restore_best_weights=True
    )
    callbacks.append(early_stopping)
    from keras.callbacks import ModelCheckpoint
    model_checkpoint_callback = ModelCheckpoint(
        filepath=f"{root_dir}/{models_path}/checkpoint.model.keras",
        monitor='val_accuracy',
        mode='max',
    )
    callbacks.append(model_checkpoint_callback)

os.makedirs(f"{root_dir}/{models_path}", exist_ok=True)
rnn_models = []
for i in range(num_models["rnn"]):

    print(f"Training rnn model {i+1} out of {num_models['rnn']} with seed {42+i}")
    print("---------------------------------------")
    set_all_seeds(42 + i)

    rnn_model = create_rnn_model_with_residual(
        window_size, numerical_features, initial_learning_rate=rnn_lr
    )

    from tensorflow.keras.utils import plot_model
    plot_model(rnn_model, to_file='gru_model.png', show_shapes=True, show_layer_names=True, dpi=96)

    history = rnn_model.fit(
        train_batches,
        train_targets,
        validation_data=(test_batches, test_targets),
        epochs=rnn_ep,
        batch_size=rnn_bs,
        callbacks=callbacks,
    )
    print("---------------------------------------")
    rnn_model.save(f"{root_dir}/{models_path}/rnn_model_seed_{i}.h5")
    rnn_models.append(rnn_model)

# rnn_model.load_model(f"{root_dir}/{models_path}/checkpoint.model.keras")
predictions = make_predictions(rnn_models, test_batches, model="nn")
print(
    f"Ensemble Mean Absolute Error: {mean_absolute_error(test_targets, predictions):.5f}"
)

prediction_df = pd.DataFrame(
    {
        "stock_id": stock_ids,
        "target": predictions.flatten(),
    }
)
weight = json.load(open(f"{root_dir}/data/weight.json"))
weight = dict(zip(range(200), weight))

prediction_df["stock_weights"] = prediction_df["stock_id"].map(weight)
prediction_df["target"] = (
    prediction_df["target"]
    - (prediction_df["target"] * prediction_df["stock_weights"]).sum()
    / prediction_df["stock_weights"].sum()
)

print(
    f"GRU Ensemble + PP Mean Absolute Error: {mean_absolute_error(test_targets, prediction_df['target']):.5f}"
)


Training rnn model 1 out of 1 with seed 42
---------------------------------------
Epoch 1/100
1064/1064 [==============================] - 27s 17ms/step - loss: 6.3853 - val_loss: 5.9022
Epoch 2/100
1064/1064 [==============================] - 16s 15ms/step - loss: 6.3516 - val_loss: 5.8858
Epoch 3/100
1064/1064 [==============================] - 16s 15ms/step - loss: 6.3389 - val_loss: 5.8801
Epoch 4/100
1064/1064 [==============================] - 16s 15ms/step - loss: 6.3304 - val_loss: 5.8751
Epoch 5/100
1064/1064 [==============================] - 16s 15ms/step - loss: 6.3244 - val_loss: 5.8730
Epoch 6/100
1064/1064 [==============================] - 16s 15ms/step - loss: 6.3201 - val_loss: 5.8684
Epoch 7/100
1064/1064 [==============================] - 16s 15ms/step - loss: 6.3164 - val_loss: 5.8696
Epoch 8/100
1064/1064 [==============================] - 16s 15ms/step - loss: 6.3134 - val_loss: 5.8707
Epoch 9/100
1064/1064 [==============================] - 16s 15ms/step - loss

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


54/54 [==============================] - 1s 16ms/step
Ensemble Mean Absolute Error: 5.86499
GRU Ensemble + PP Mean Absolute Error: 5.86509


In [16]:
prediction_df.to_parquet(f"{root_dir}/data/prediction_gru.parquet")